In [1]:
import numpy as np
import cupy as cp

from numbers import Number, Integral
from typing import Optional, Union, List, Tuple

ArrayOnCPU = np.ndarray
ArrayOnGPU = cp.ndarray
ArrayOnCPUOrGPU = Union[cp.ndarray, np.ndarray]

In [2]:
from abc import ABCMeta, abstractmethod
from sklearn.base import BaseEstimator
from ksig import utils

class Kernel(BaseEstimator, metaclass=ABCMeta):
    """Base class for Kernels.

    Warning: This class should not be used directly.
    Use derived classes instead.
    """

    def fit(self, X : ArrayOnCPUOrGPU, y : Optional[ArrayOnCPUOrGPU] = None):
        return self

    @abstractmethod
    def _K(self, X : ArrayOnGPU, Y : Optional[ArrayOnGPU] = None) -> ArrayOnGPU:
        pass

    @abstractmethod
    def _Kdiag(self, X : ArrayOnGPU) -> ArrayOnGPU:
        pass

    def __call__(self, X : ArrayOnCPUOrGPU, Y : Optional[ArrayOnCPUOrGPU] = None, diag : bool = False, return_on_gpu : bool = False) -> ArrayOnCPUOrGPU:
        X = cp.asarray(X)
        Y = cp.asarray(Y) if Y is not None else None
        if diag:
            K = self._Kdiag(X)
        else:
            K =  self._K(X, Y)
        if not return_on_gpu:
            K = cp.asnumpy(K)
        return K

# ----------------------------------------------------------------------------------------------------------------------------------------------------------------

class LinearKernel(Kernel):
    """Class for linear (static) kernel."""

    def __init__(self, sigma : float = 1.0) -> None:
        self.sigma = utils.check_positive_value(sigma, 'sigma')

    def _K(self, X : ArrayOnGPU, Y : Optional[ArrayOnGPU] = None) -> ArrayOnGPU:
        return self.sigma**2 * utils.matrix_mult(X, Y, transpose_Y=True)

    def _Kdiag(self, X : ArrayOnGPU) -> ArrayOnGPU:
        return self.sigma**2 * utils.squared_norm(X, axis=-1)

In [3]:
def multi_cumsum(M : ArrayOnGPU, exclusive : bool = False, axis : int = -1) -> ArrayOnGPU:
    """Computes the exclusive cumulative sum along a given set of axes.

    Args:
        K (cp.ndarray): A matrix over which to compute the cumulative sum
        axis (int or iterable, optional): An axis or a collection of them. Defaults to -1 (the last axis).
    """

    ndim = M.ndim
    axis = [axis] if cp.isscalar(axis) else axis
    axis = [ndim+ax if ax < 0 else ax for ax in axis]

    # create slice for exclusive cumsum (slice off last element along given axis then pre-pad with zeros)
    if exclusive:
        slices = tuple(slice(-1) if ax in axis else slice(None) for ax in range(ndim))
        M = M[slices]

    # compute actual cumsums
    for ax in axis:
        M = cp.cumsum(M, axis=ax)

    # pre-pad with zeros along the given axis if exclusive cumsum
    if exclusive:
        pads = tuple((1, 0) if ax in axis else (0, 0) for ax in range(ndim))
        M = cp.pad(M, pads)

    return M

In [4]:
def signature_kern_higher_order(M : ArrayOnGPU, n_levels : int, order : int, difference : bool = True, return_levels : bool = False) -> ArrayOnGPU:
    """
    Computes the signature kernel matrix with higher-order embedding into the tensor algebra.
    """

    if difference:
        M = cp.diff(cp.diff(M, axis=1), axis=-1)

    if M.ndim == 4:
        n_X, n_Y = M.shape[0], M.shape[2]
        K = cp.ones((n_X, n_Y), dtype=M.dtype)
    else:
        n_X = M.shape[0]
        K = cp.ones((n_X,), dtype=M.dtype)

    if return_levels:
        K = [K, cp.sum(M, axis=(1, -1))]
    else:
        K += cp.sum(M, axis=(1, -1))

    R = cp.copy(M)[None, None, ...]
    for i in range(1, n_levels):
        d = min(i+1, order)
        R_next = cp.empty((d, d) + M.shape, dtype=M.dtype)
        R_next[0, 0] = M * multi_cumsum(cp.sum(R, axis=(0, 1)), exclusive=True, axis=(1, -1))
        for r in range(1, d):
            R_next[0, r] = 1./(r+1) * M * multi_cumsum(cp.sum(R[:, r-1], axis=0), exclusive=True, axis=1)
            R_next[r, 0] = 1./(r+1) * M * multi_cumsum(cp.sum(R[r-1, :], axis=0), exclusive=True, axis=-1)
            for s in range(1, d):
                R_next[r, s] = 1./((r+1)*(s+1)) * M * R[r-1, s-1]
        R = R_next
        if return_levels:
            K.append(cp.sum(R, axis=(0, 1, 3, -1)))
        else:
            K += cp.sum(R, axis=(0, 1, 3, -1))

    return cp.stack(K, axis=0) if return_levels else K

In [32]:
# simulate geometric Brownian motion paths
n_paths = 4
n_steps = 50 - 1

mu_x = 0.1
sigma_x = 0.2
mu_y = 0.2
sigma_y = 0.2
dt = 0.01
X = np.exp((mu_x - 0.5 * sigma_x**2) * dt + sigma_x * np.sqrt(dt) * np.random.randn(n_paths, n_steps))
Y = np.exp((mu_y - 0.5 * sigma_y**2) * dt + sigma_y * np.sqrt(dt) * np.random.randn(n_paths, n_steps))
X = np.cumprod(X, axis=1)
Y = np.cumprod(Y, axis=1)
X = np.concatenate([np.ones((n_paths, 1)), X], axis=1)
Y = np.concatenate([np.ones((n_paths, 1)), Y], axis=1)
X = X[..., np.newaxis]
Y = Y[..., np.newaxis]
X = cp.asarray(X)
Y = cp.asarray(Y)
X.shape, Y.shape

((4, 50, 1), (4, 50, 1))

In [33]:
static_kernel = LinearKernel()

# M = static_kernel(X, diag=True, return_on_gpu=True)

M_X = static_kernel(X.reshape((-1, X.shape[-1])), return_on_gpu=True).reshape((X.shape[0], X.shape[1], X.shape[0], X.shape[1]))
M_Y = static_kernel(Y.reshape((-1, Y.shape[-1])), return_on_gpu=True).reshape((Y.shape[0], Y.shape[1], Y.shape[0], Y.shape[1]))
M_XY = static_kernel(X.reshape((-1, X.shape[-1])), Y.reshape((-1, Y.shape[-1])), return_on_gpu=True).reshape((X.shape[0], X.shape[1], Y.shape[0], Y.shape[1]))
M_X.shape, M_Y.shape, M_XY.shape

((4, 50, 4, 50), (4, 50, 4, 50), (4, 50, 4, 50))

In [34]:
n_levels = 5
order = 5
sig_levels = signature_kern_higher_order(M_XY, n_levels, order, return_levels=True)
sig = signature_kern_higher_order(M_XY, n_levels, order, return_levels=False)
sig_levels.shape, sig.shape

((6, 4, 4), (4, 4))

In [35]:
def is_symmetric(X):
    if X.ndim == 2:
        return cp.allclose(X, X.T)
    else:
        raise ValueError('X must be a 2D matrix')

is_symmetric(sig)

array(False)

In [36]:
sig

array([[0.99468976, 1.00585679, 1.00666171, 1.01360896],
       [0.99984128, 1.00017458, 1.00019853, 1.00040489],
       [1.00472597, 0.99481501, 0.99410465, 0.98799583],
       [1.00436833, 0.99520649, 0.99454969, 0.98890074]])

In [41]:
norms = signature_kern_higher_order(M_Y, n_levels, order, return_levels=True)
norms.shape

(6, 4, 4)

In [42]:
for i in range(n_levels + 1):
    print(cp.sqrt(norms[i].diagonal()))


[1. 1. 1. 1.]
[0.07637501 0.084001   0.09552646 0.19481062]
[0.00291657 0.00352808 0.00456265 0.01897559]
[7.42510623e-05 9.87875415e-05 1.45284673e-04 1.23221546e-03]
[1.41773149e-06 2.07456316e-06 3.46963263e-06 6.00121650e-05]
[2.16558528e-08 3.48530776e-08 6.62883444e-08 2.33820144e-06]


In [28]:
cp.allclose(sig_levels.sum(axis=0), sig)

array(True)